In [72]:
import sqlite3
from sqlite3 import Error
import random
import names 
import pandas as pd


# 1. Create a schema #
I'll define a schema for the chickens and eggs table, and implement those in sqlite.

In [179]:
## 1. Create a schema (and build the database in SQLite)

# create or connect to the database
db_path = 'chickens.db'
conn = sqlite3.connect(db_path)

# create a cursor
c = conn.cursor()

# define tables as sql code strings
sql_create_chickens_table = """ CREATE TABLE chickens (  
                                    chicken_id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    sex text NOT NULL,
                                    feather_color text NOT NULL,
                                    favorite_song text NOT NULL,
                                    from_egg_id integer NULLABLE,
                                    FOREIGN KEY (from_egg_id) REFERENCES eggs(egg_id)
                                ); """

sql_create_eggs_table = """ CREATE TABLE IF NOT EXISTS eggs (
                                egg_id integer PRIMARY KEY,
                                f_parent_chicken_id integer NOT NULL,
                                m_parent_chicken_id integer NOT NULL,
                                incubation_location text NOT NULL,
                                is_by_window boolean NOT NULL,
                                FOREIGN KEY (f_parent_chicken_id) REFERENCES chickens (chicken_id),
                                FOREIGN KEY (m_parent_chicken_id) REFERENCES chickens (chicken_id)
                            );"""

# create the tables in the database

# create chickens table
c.execute("DROP TABLE IF EXISTS chickens;")
c.execute(sql_create_chickens_table)
# create eggs table
c.execute("DROP TABLE IF EXISTS eggs;")
c.execute(sql_create_eggs_table)


# 2. Generate Data # 
I'll populate those tables with some random data, then discuss how realistic those data are.

In [180]:
# generate random data in the chickens and eggs tables

# functions to generate random data   
def random_name(sex:str = "Hen") -> str:
    """Generate a random chicken name
    we're going for realism here, so we'll use a python package called names
    that gets it's data from the US census.  I haven't found a package for chicken names, 
    so human names will have to do.
    """
    if sex=="Hen":
        return names.get_first_name(gender="female")
    if sex=="Rooster":
        return names.get_first_name(gender="male")
    else:
        return names.get_first_name()

def random_sex(hen_weight:int = 10) -> str:
    """Generate a random chicken sex, "Rooster" or "Hen" 
    There should be more hens than roosters by a factor of hen_weight
    """
    return random.choices(["Rooster", "Hen"], weights=[1, hen_weight])[0]

def random_feather_color():
    """Generate a random feather color"""
    return random.choice(["Black", "White", "Brown", "Red", "Speckled"])

def random_song():
    """Generate a random favorite song"""
    return random.choice(["Egg Head -- Benny Goodman", "I'm Putting All My Eggs In One Basket -- Louis Armstrong", "Egg Raid On Mojo -- Beastie Boys", "Pork Roll Egg and Cheese -- Ween", "Eggs And Sausage -- Tom Waits", "They Punctured My Yolk -- Flaming Lips", "Egg Man -- Beastie Boys", "Egg -- Smashing Pumpkins", "Hammerin Eggs -- Goo Goo Dolls"])

def random_egg():
    """Generate a random from_egg_id"""
    return random.randint(1, 1000) # for now

def random_chicken() -> str:
    """Generate a random chicken"""
    sex = random_sex()
    name = random_name(sex)
    feather_color = random_feather_color()
    favorite_song = random_song()
    from_egg_id = random_egg()

    sql = f"""INSERT INTO chickens(name, sex, feather_color, favorite_song, from_egg_id)
            VALUES("{name}","{sex}","{feather_color}","{favorite_song}", "{from_egg_id}");
            """

    return sql

# generate 1000 chickens
for i in range(1000):
    sql = random_chicken()
    print
    conn.execute(sql)


# generate fake egg data 
def random_parent(sex:str) -> int:
    """Generate a random parent chicken_id"""
    c.execute(f"""select chicken_id from chickens where sex = "{sex}" """)
    ids = [row[0] for row in c.fetchall()]
    return random.choice(ids)

def random_incubation_location() -> int:
    """Generate a random incubation location
    nesting locationsa are numbered 1-100"""
    return random.randint(1,100)

def random_is_by_window() -> bool:
    """Generate a random bool value is_by_window"""
    return random.choices([True, False], weights=[1, 3])[0]

def random_egg_clutch():
    """ generate a clutch of between 1 and 6 eggs
    clutches share a location and maternal parent
    paternal parent can be different for each egg"""
    clutch_size = random.randint(1, 6)
    f_parent_id = random_parent("Hen")
    incubation_location = random_incubation_location()
    is_by_window = random_is_by_window()
    for i in range(clutch_size):
        m_parent_id = random_parent("Rooster")
        sql = f"""INSERT INTO eggs(f_parent_chicken_id, m_parent_chicken_id, incubation_location, is_by_window)
                VALUES("{f_parent_id}","{m_parent_id}","{incubation_location}", "{is_by_window}");
                """
        conn.execute(sql)
    return clutch_size

# generate 10,000 eggs
def generate_eggs(n:int = 10000):
    counter = 0
    while counter < n:
        clutch_size = random_egg_clutch()
        counter += clutch_size
    print(f"generated {counter} eggs")

generate_eggs()

# take a look 
c.execute("""select * from chickens limit 5""")
rows = c.fetchall()
print(" -- chickens -- ")
for row in rows:
    print(row)

c.execute("""select * from eggs limit 5""")
rows = c.fetchall()
print(" -- eggs -- ")
print("f_parent_chicken_id, m_parent_chicken_id, incubation_location, is_by_window")
for row in rows:
    print(row)

generated 10000 eggs
 -- chickens -- 
(1, 'Nikki', 'Hen', 'Black', 'Egg -- Smashing Pumpkins', 369)
(2, 'Bettie', 'Hen', 'Brown', 'Egg Man -- Beastie Boys', 616)
(3, 'Elizabeth', 'Hen', 'White', 'Hammerin Eggs -- Goo Goo Dolls', 362)
(4, 'Billy', 'Rooster', 'Brown', 'Hammerin Eggs -- Goo Goo Dolls', 748)
(5, 'Kristi', 'Hen', 'White', 'Egg -- Smashing Pumpkins', 811)
 -- eggs -- 
f_parent_chicken_id, m_parent_chicken_id, incubation_location, is_by_window
(1, 729, 482, '79', 'False')
(2, 350, 742, '61', 'False')
(3, 536, 906, '37', 'False')
(4, 536, 741, '37', 'False')
(5, 536, 297, '37', 'False')


In [165]:
# take a look at the chickens as a dataframe this time, so its easier to read
chickens = pd.read_sql_query("select * from chickens", conn)
chickens.head(40)

,chicken_id,name,sex,feather_color,favorite_song,from_egg_id
0,1,Vernice,Hen,Black,Egg Head -- Benny Goodman,737
1,2,Naomi,Hen,Brown,Pork Roll Egg and Cheese -- Ween,783
2,3,Maud,Hen,Black,They Punctured My Yolk -- Flaming Lips,521
3,4,Sylvia,Hen,Black,Egg Man -- Beastie Boys,246
4,5,Claudine,Hen,White,Hammerin Eggs -- Goo Goo Dolls,935
5,6,Aubrey,Rooster,Speckled,Egg -- Smashing Pumpkins,739
6,7,Patricia,Hen,White,Egg Head -- Benny Goodman,633
7,8,Patrick,Rooster,Red,Egg Head -- Benny Goodman,78
8,9,Jada,Hen,White,Egg -- Smashing Pumpkins,774
9,10,Betsy,Hen,White,Egg Raid On Mojo -- Beastie Boys,349


Our data is looking pretty realistic.  The forensic data scientist / ag inspector may construct a parantage graph form this data, and it could look unrealistic. For example, it could contain some nonsensical loops, in which parents are decendents of themselves. It would take us a lot of work to generate a realistic graph, so it's a risk we are willing to take.  

The inspector could use a couple of common techniques for detecting manually fabricated data (entered by a person, but falsified). These have been used to assess the valitdity of clinical trials: 

1. Compare the distributions of our data to a baseline assumption derived from a broader data set. Presumably, the inspector has access to data from other comperable chicken farms.  They could compare the distributions of feather color, favorite songs, etc from those farms with our distributions, and use a simple statistical test (like Chi-Square) to calculate a probability that the difference between our distibution and the broader distribution is due to chance.  At a very low probabilty, they might ask questions about our data. 

2. They may look for irregularities in numbers caused by human bias in faslified data entry. For example, they could examine the distribution of the last digit in the parent ids.  If data was falsified by hand, it could be significantly biased towards some last digits, where if it was real, it would be fairly evenly distributed.  Our data is randomized, so we won't have that problem. 

# 3. Name Tags for Chickens # 

In [178]:
# query the database to get a table of chickens with their parents and grandparents' names
# this is something we could materialize as a table using dbt so that we can re-use it for reporting

sql = """with chickens_parents as (
        select
            chickens.*,
            eggs.f_parent_chicken_id as f_parent_id,
            eggs.m_parent_chicken_id as m_parent_id,
            eggs.incubation_location
        from chickens
        left join eggs on chickens.from_egg_id = eggs.egg_id
        )
        , chickens_parents_names as (
        select 
            chickens.chicken_id, 
            chickens.name,
            chickens.sex,
            chickens.feather_color,
            chickens.favorite_song,
            chickens.incubation_location,

            fathers.name as father_name,
            mothers.name as mother_name,

            chickens.m_parent_id,
            fathers.m_parent_id as paternal_grandfather_id,
            fathers.f_parent_id as paternal_grandmother_id,
            chickens.f_parent_id,
            mothers.m_parent_id as maternal_grandfather_id,
            mothers.f_parent_id as maternal_grandmother_id
        from chickens_parents as chickens
        left join chickens_parents as fathers on chickens.m_parent_id = fathers.chicken_id
        left join chickens_parents as mothers on chickens.f_parent_id = mothers.chicken_id
        )
        , grandparents as (select 
            parents.chicken_id,
            parents.name,
            parents.sex,
            parents.feather_color,
            parents.favorite_song,
            parents.incubation_location,

            parents.father_name,
            parents.mother_name,
            paternal_grandfathers.name as paternal_grandfather_name,
            paternal_grandmothers.name as paternal_grandmother_name,
            maternal_grandfathers.name as maternal_grandfather_name,
            maternal_grandmothers.name as maternal_grandmother_name,

            parents.paternal_grandfather_id,
            parents.paternal_grandmother_id,
            parents.maternal_grandfather_id,
            parents.maternal_grandmother_id

        from chickens_parents_names as parents
        left join chickens_parents_names as paternal_grandfathers on parents.paternal_grandfather_id = paternal_grandfathers.chicken_id
        left join chickens_parents_names as paternal_grandmothers on parents.paternal_grandmother_id = paternal_grandmothers.chicken_id
        left join chickens_parents_names as maternal_grandfathers on parents.maternal_grandfather_id = maternal_grandfathers.chicken_id
        left join chickens_parents_names as maternal_grandmothers on parents.maternal_grandmother_id = maternal_grandmothers.chicken_id
        )
        , randomized_cousins as (
            select
                grandparents.chicken_id,
                cousins.chicken_id as cousin_id,
                cousins.name as cousin_name,
                row_number() over (partition by grandparents.chicken_id order by random()) as cousin_number
            from grandparents
            inner join grandparents as cousins 
                on grandparents.paternal_grandfather_id = cousins.paternal_grandfather_id
                    or grandparents.paternal_grandmother_id = cousins.paternal_grandmother_id
                    or grandparents.maternal_grandfather_id = cousins.maternal_grandfather_id
                    or grandparents.maternal_grandmother_id = cousins.maternal_grandmother_id
                    or grandparents.paternal_grandfather_id = cousins.maternal_grandfather_id
                    or grandparents.paternal_grandmother_id = cousins.maternal_grandmother_id
        )
        select 
            grandparents.chicken_id, 
            grandparents.name,
            grandparents.sex,
            grandparents.feather_color,
            grandparents.favorite_song,
            grandparents.mother_name,
            grandparents.father_name,
            grandparents.paternal_grandfather_name,
            grandparents.paternal_grandmother_name,
            grandparents.maternal_grandfather_name,
            grandparents.maternal_grandmother_name,
            grandparents.incubation_location,
            cousins.cousin_name
        from grandparents
        left join randomized_cousins as cousins on grandparents.chicken_id = cousins.chicken_id
        where cousins.cousin_number = 1
        limit 5
        """ 

# print out name tags 

c.execute(sql)
rows = c.fetchall()
for row in rows:
    print(f"""
    Name: {row[1]}
    Sex: {row[2]}
    Favorite Song: {row[4]}
    Parents: {row[5]} and {row[6]}
    Grandparents: {row[7]}, {row[8]}, {row[9]} and {row[10]}
    I was incubated at location: {row[11]}
    One of my First Cousins: {row[12]}
    """)


    Name: Vernice
    Sex: Hen
    Favorite Song: Egg Head -- Benny Goodman
    Parents: Gina and Larry
    Grandparents: Frank, Amanda, Kevin and Michal
    I was incubated at location: 33
    One of my First Cousins: Carolyn
    

    Name: Naomi
    Sex: Hen
    Favorite Song: Pork Roll Egg and Cheese -- Ween
    Parents: Jacqueline and Adam
    Grandparents: Sung, Debra, Brent and Helga
    I was incubated at location: 86
    One of my First Cousins: William
    

    Name: Maud
    Sex: Hen
    Favorite Song: They Punctured My Yolk -- Flaming Lips
    Parents: Amy and Thomas
    Grandparents: Arthur, Effie, Kenneth and Essie
    I was incubated at location: 30
    One of my First Cousins: Cheryl
    

    Name: Sylvia
    Sex: Hen
    Favorite Song: Egg Man -- Beastie Boys
    Parents: Anna and Joseph
    Grandparents: Tim, Johanna, William and Edna
    I was incubated at location: 5
    One of my First Cousins: Cynthia
    

    Name: Claudine
    Sex: Hen
    Favorite Song: Ham

So there we have the nametags.  Note that I departed slightly from the directions, giving each chicken's incubation location, but not the incubation location of their parents and grandparents.  This is an example of how I would go back to the person requesting the data and ask, are you sure you want these tags to have incubation location for all parents and grandparents? What format would work for you? How do you imagine using it? It's not hard to do, but would require joining each grandparents table to the eggs table, and I'd want to varify that I understand the need.  In the process, I might find that the data user needs something different -- the number of eggs laid to date, the feather color of parents, etc. 

I'd love to make a bonus dashboard, but I've got a dozen of these assessments to do this week.  Looking forward to talking.  